In [4]:
## knjižnice:
import pandas as pd
import numpy as np
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [5]:
import statsmodels.api as sm
from statsmodels.graphics import tsaplots
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm

In [6]:
## podatki, ki so nam jih poslali:
podatki = pd.read_csv(os.path.join('C:/Users/Uporabnik/OneDrive/Namizje/SOLA/MAGISTERIJ/MATEMATIKA Z RACUNALNIKOM/GospodinjskiOdjemalci.csv'), sep=";")
# Malo boljši format:
podatki['DateTimeStartUTC'] = pd.to_datetime(podatki['DateTimeStartUTC'], format='%d.%m.%Y %H:%M')
podatki['DateTimeStartCET'] = pd.to_datetime(podatki['DateTimeStartCET'], format='%d.%m.%Y %H:%M')

podatki = podatki[['DateTimeStartUTC', 'Odjem ACT', 'Temperatura ACT', 'Temperatura FC', 'Sevanje ACT', 'Sevanje FC']]

podatki.set_index('DateTimeStartUTC', inplace=True)
podatki['Year'] = podatki.index.year
podatki['Month'] = podatki.index.month
podatki['Day'] = podatki.index.date

podatki = podatki.asfreq('15T')

podatki['Odjem ACT'] = podatki['Odjem ACT'] * 1000000000 # da dobim MWh (bo lepše pri logaritmu tega stolpce v nadaljevanju)

podatki

,Odjem ACT,Temperatura ACT,Temperatura FC,Sevanje ACT,Sevanje FC,Year,Month,Day
DateTimeStartUTC,,,,,,,,
2021-10-31 23:00:00,9508.67,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-10-31 23:15:00,9168.37,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-10-31 23:30:00,8860.11,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-10-31 23:45:00,8510.97,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-11-01 00:00:00,8332.21,3.45,5.300,0.0,0.0,2021,11,2021-11-01
...,...,...,...,...,...,...,...,...
2024-02-29 21:45:00,12005.50,6.80,7.475,0.0,0.0,2024,2,2024-02-29
2024-02-29 22:00:00,11336.40,6.65,7.300,0.0,0.0,2024,2,2024-02-29
2024-02-29 22:15:00,10796.70,6.50,7.300,0.0,0.0,2024,2,2024-02-29


In [ ]:
# multivariatni modeli

In [ ]:
# 1. Long Short Term Memory (LSTM)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

TypeError: unhashable type: 'list'

In [44]:
df_for_training = podatki.loc['2023-02-24':'2023-02-28', ['Odjem ACT', 'Temperatura FC']]
df_for_testing = podatki.loc['2023-03-01':'2023-03-05', ['Odjem ACT', 'Temperatura FC']]

In [45]:
# Scaling the data:

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
df_for_training_scaled = scaler.fit_transform(df_for_training)
df_for_testing_scaled=scaler.transform(df_for_testing)

df_for_training_scaled

array([[0.08204331, 0.39751553],
       [0.07638543, 0.38664596],
       [0.06677782, 0.3757764 ],
       [0.05890318, 0.36490683],
       [0.0554085 , 0.35403727],
       [0.04682532, 0.3431677 ],
       [0.05041058, 0.33229814],
       [0.0540505 , 0.32142857],
       [0.05538531, 0.31055901],
       [0.05290236, 0.29968944],
       [0.05656935, 0.28881988],
       [0.05699348, 0.27795031],
       [0.07383677, 0.26708075],
       [0.08702467, 0.26086957],
       [0.0950462 , 0.25465839],
       [0.11536705, 0.2484472 ],
       [0.14707213, 0.24223602],
       [0.17565143, 0.24223602],
       [0.19517924, 0.24223602],
       [0.21444749, 0.24223602],
       [0.24420862, 0.24223602],
       [0.27220805, 0.24068323],
       [0.29417684, 0.23913043],
       [0.31475393, 0.23757764],
       [0.32732883, 0.23602484],
       [0.34077629, 0.24689441],
       [0.34350996, 0.25776398],
       [0.33507147, 0.26863354],
       [0.32299361, 0.27950311],
       [0.31911124, 0.31832298],
       [0.

In [46]:
# spliting the data in X and Y:

def createXY(dataset, n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
            dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
            dataY.append(dataset[i,0])
    return np.array(dataX), np.array(dataY)

trainX, trainY = createXY(df_for_training_scaled, 200)
testX, testY = createXY(df_for_testing_scaled, 200)

print("trainX Shape-- ",trainX.shape)
print("trainY Shape-- ",trainY.shape)

print("testX Shape-- ",testX.shape)
print("testY Shape-- ",testY.shape)

trainX Shape--  (280, 200, 2)
trainY Shape--  (280,)
testX Shape--  (280, 200, 2)
testY Shape--  (280,)


In [53]:
# traing the model:

from sklearn.model_selection import GridSearchCV
# from keras.wrappers.scikit_learn import KerasRegressor
# from keras.wrappers.scikit_learn import KerasRegressor
# import scikeras
# from scikeras.wrappers import KerasRegressor 
# from sklearn_r2_score import KerasRegressor 
from scikeras.wrappers import KerasClassifier

# def build_model(optimizer):
#     grid_model = Sequential()
#     grid_model.add(LSTM(50,return_sequences=True,input_shape=(200,2)))
#     grid_model.add(LSTM(50))
#     grid_model.add(Dropout(0.2))
#     grid_model.add(Dense(1))
#     grid_model.compile(loss = 'mse',optimizer = optimizer)
#     return grid_model

# grid_model = KerasRegressor(build_fn=build_model,verbose=1,validation_data=(testX,testY))
# parameters = {'batch_size' : [16,20],
#               'epochs' : [8,10],
#               'optimizer' : ['adam','Adadelta'] }
# grid_search  = GridSearchCV(estimator = grid_model,
#                             param_grid = parameters,
#                             cv = 2)

TypeError: unhashable type: 'list'